In [43]:
%config Completer.use_jedi = False
%matplotlib inline
import calendar
import datetime
import geopandas as gpd
import itertools
import libpysal
import mapclassify
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import plotly_express as px
import seaborn as sns
import statsmodels.api as sm
import warnings
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
from matplotlib.pyplot import figure
from pandas import to_datetime
# from fbprophet import Prophet
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.stattools import adfuller
from zipfile import ZipFile

#warnings.filterwarnings('ignore')
#warnings.simplefilter('ignore')

In [44]:
pd.set_option('display.max_rows', 50000)
pd.set_option('display.max_colwidth', None)

year_2014 = datetime.strptime('2014-3-15', '%Y-%m-%d')
year_2017 = datetime.strptime('2017-3-15', '%Y-%m-%d')
year_2019 = datetime.strptime('2019-3-15', '%Y-%m-%d')
year_2020 = datetime.strptime('2020-3-15', '%Y-%m-%d')
graph_end = datetime.strptime('2021-07-31', '%Y-%m-%d')

# Daily boarding totals
board_df = pd.read_csv('/kaggle/input/ridership-data/Daily_Boarding_Totals.csv',index_col='service_date',parse_dates=True)
board_df = board_df.drop(['day_type','rail_boardings','total_rides'],axis=1).rename(columns={'bus':'riders'})
board_df.index.names = ['date']
board_df = board_df.truncate(before=year_2014)
board_df.index.freq='D'

# Daily boarding totals by route
routes_df = pd.read_csv('/kaggle/input/ridership-data/Bus_Routes_Monthly_DayType_Averages_Totals.csv')
routes_df = routes_df.drop(["Avg_Weekday_Rides","Avg_Saturday_Rides","Avg_Sunday-Holiday_Rides"], axis=1)
routes_df = routes_df.rename(columns={'Month_Beginning': 'date', 'MonthTotal': 'riders'})
routes_df['date'] = pd.to_datetime(routes_df['date'])
routes_df = routes_df.set_index('date').truncate(before=year_2014).reset_index()
num_months = (graph_end.year - year_2014.year) * 12 + (graph_end.month - year_2020.month)
missing_val = routes_df.groupby(['route'], as_index=False).count()
missing_val.drop(missing_val[missing_val['riders'] >= num_months].index, inplace = True)
routes_df = routes_df.loc[~routes_df['route'].isin(missing_val['route'])]

In [45]:
# Daily riders over a three-year pre-pandemic period and the pandemic period
def graph(y):
    fig, ax = plt.subplots(figsize=(11,6), dpi=100)
    ax.plot(y.truncate(before=year_2017),
            marker='.', markersize=5, linestyle='-', linewidth=0.5, label='Daily')
    ax.plot(y.truncate(before=year_2017).resample('M').mean(),
            marker='o', markersize=8, linestyle='-', label='Monthly Mean')
    ax.set_ylabel('Daily Bus Riders')
    ax.legend();
    
graph(board_df['riders'])

In [46]:
# Augmented Dickey-Fuller Test
def adf_test(y):
    print(' > Is the raw data stationary ?')
    print('Date range: ', y.index[0].date(), '-', y.index[-1].date())
    y = y['riders']
    dftest = adfuller(y.dropna(), autolag='AIC')
    print('Test statistic = {:.3f}'.format(dftest[0]))
    print('P-value = {:.3f}'.format(dftest[1]))
    print('Critical values :')
    for k, v in dftest[4].items():
        print('\t{}: {} - The data is {}stationary with {}% confidence'
              .format(k, v, 'not' if v<dftest[0] else '', 100-int(k[:-1])))
    
adf_test(board_df.truncate(after=year_2020))

# The data is stationary, therefore, additional decomposition is unneeded.

In [47]:
# fbprophet has a bug that spits out reams of print statements without this hotfix
# from https://stackoverflow.com/questions/11130156/suppress-stdout-stderr-print-from-python-functions
class suppress_stdout_stderr(object):
    def __init__(self):
        self.null_fds = [os.open(os.devnull, os.O_RDWR) for x in range(2)]
        self.save_fds = (os.dup(1), os.dup(2))

    def __enter__(self):
        os.dup2(self.null_fds[0], 1)
        os.dup2(self.null_fds[1], 2)

    def __exit__(self, *_):
        os.dup2(self.save_fds[0], 1)
        os.dup2(self.save_fds[1], 2)
        os.close(self.null_fds[0])
        os.close(self.null_fds[1])

In [48]:
def prophet_forecast(dataframe, pred_begin, frequency):
    train_begin = pred_begin-relativedelta(years=3)
    
    df = dataframe.reset_index()
    df.columns = ['ds', 'y']
    df['ds']= to_datetime(df['ds'])

    train = df.set_index('ds').truncate(before=train_begin, after=pred_begin).reset_index()

    if frequency == 'M' or frequency == 'MS':
        model = Prophet(yearly_seasonality=True,weekly_seasonality=False,daily_seasonality=False)
    else:
        model = Prophet(yearly_seasonality=True,weekly_seasonality=True,daily_seasonality=True)
        print('Training period:   ',train_begin.date(),' - ',pred_begin.date())
        print('Prediction period: ',pred_begin.date(),' - ',(pred_begin+relativedelta(years=1)).date())

    model.fit(train)

    pred = list()
    for date in pd.date_range(start=pred_begin, end=graph_end, freq=frequency):
        pred.append([date.strftime("%Y-%m-%d")])
    pred = pd.DataFrame(pred)
    pred.columns = ['ds']
    pred['ds'] = to_datetime(pred['ds'])

    forecast = model.predict(pred)

    #model.plot(forecast)
    #plt.show()
    
    return forecast, df


def mean_abs_error(forecast, df):
    mae = mean_absolute_error(df['y'][-len(forecast):].values,
                              forecast['yhat'].values)
    print('Mean Absolute Error: %.3f' % mae)

def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    print('Mean Absolute %% Error:  %.3f' % mape)
    return

def graph_forecast(forecast, df):
    y_true = df.set_index('ds')
    y_pred = forecast[['ds','yhat']].set_index('ds').squeeze()
    fig, ax = plt.subplots(figsize=(11,6), dpi=100)
    y_true = y_true.squeeze()
    y_true = y_true.truncate(before=year_2017)
    ax.plot(y_true, marker='.', markersize=5, linestyle='-', linewidth=0.5, label='Actual')
    ax.plot(y_pred, linewidth=0.8, label='Predicted')
    ax.set_ylabel('Daily Bus Riders')
    ax.legend();

In [73]:
#with suppress_stdout_stderr():
#    forecast, returned_df = prophet_forecast(board_df.copy(),year_2019,'D')
forecast = pd.read_csv('/kaggle/input/prophetdata/forecast1.csv',parse_dates=True)
forecast['ds'] = pd.to_datetime(forecast['ds'])
returned_df = pd.read_csv('/kaggle/input/prophetdata/returned_df1.csv')
returned_df['ds'] = pd.to_datetime(returned_df['ds'])

forecast = forecast.set_index('ds').truncate(after=year_2020).reset_index()
y_true = returned_df.set_index('ds').truncate(before=year_2019,after=year_2020).reset_index()
mean_abs_error(forecast,y_true)
mean_absolute_percentage_error(forecast['yhat'],y_true['y'])
graph_forecast(forecast,returned_df)

In [74]:
#with suppress_stdout_stderr():
#    forecast, returned_df = prophet_forecast(board_df.copy(),year_2020,'D')
forecast = pd.read_csv('/kaggle/input/prophetdata/forecast2.csv',parse_dates=True)
forecast['ds'] = pd.to_datetime(forecast['ds'])
returned_df = pd.read_csv('/kaggle/input/prophetdata/returned_df2.csv')
returned_df['ds'] = pd.to_datetime(returned_df['ds'])    
    
y_true = returned_df.set_index('ds').truncate(before=year_2020).reset_index()
mean_abs_error(forecast,y_true)
mean_absolute_percentage_error(forecast['yhat'],y_true['y'])
graph_forecast(forecast,returned_df)

In [9]:
# Line graph of bus riders over time, by route
routes_graph = routes_df.set_index('date').truncate(before=year_2017-relativedelta(months=1))
routes_graph = routes_graph.reset_index().groupby('route')                
figure(figsize=(11,6), dpi=100)
for name, group in routes_graph:
    plt.plot(group['date'],group['riders'],label=name)

In [10]:
# NOTE: The call to route_forecast is commented out due to an
# execution time of about 5 minutes on this machine. A CSV file
# with the same contents has been provided for ease of use.

riders_by_route = routes_df.set_index('date')
route_diff = []


def percentage_change(col1, col2):
    return round(((col2 - col1) / col1) * 100)


def route_forecast(route_list):
    for i in route_list:
        route_loc = riders_by_route.loc[riders_by_route['route'] == i]
        routename = route_loc['routename'][0]
        forecast, returned_df = prophet_forecast(route_loc.drop(columns=['route','routename']),year_2020,'MS')

        forecast = forecast[['ds','yhat']].set_index('ds')
        returned_df = returned_df.iloc[-16:].set_index('ds')
        forecast = forecast.merge(returned_df, how='inner', left_index=True, right_index=True).sum()

        route_diff.append([i, routename, round(forecast['yhat']), round(forecast['y'])])

    routes_df2 = pd.DataFrame(route_diff, columns=['Route','Route Name','Predicted','Actual'])
    routes_df2['Rider % Change'] = percentage_change(routes_df2['Predicted'], routes_df2['Actual'])
    return routes_df2

#route_df = route_forecast(sorted(routes_df['route'].unique()))                      # swap which of these lines 
route_df = pd.read_csv('/kaggle/input/ridership-data/route_forecast_results.csv')  # is commented out
print(route_df.set_index('Route'))

In [11]:
route_df['clusters'] = route_df['Rider % Change'].loc[route_df['Rider % Change'] < 0]

plt.figure(figsize=(10,4), dpi=100)
ax = sns.histplot(route_df['clusters'], kde=False)
ax.set(xlabel='Percentage Loss in Ridership', ylabel='Number of Routes')
plt.show()

In [12]:
kmz = ZipFile('/kaggle/input/ridership-data/CTA_BusRoutes.kmz', 'r')
kml = kmz.open('doc.kml', 'r')

gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
gdf = gpd.read_file(kml, driver='KML')
gdf = gdf.rename(columns={'Name': 'Route'})
gdf = gdf.merge(route_df, on='Route', how='inner')#.set_index('Route').sort_index()
gdf = gpd.GeoDataFrame(gdf)

legend = mapclassify.Quantiles(gdf['Rider % Change'], k=5)
mapping = dict([(i,s) for i,s in enumerate(legend.get_legend_classes())])

In [13]:
f, ax = plt.subplots(1, figsize=(10,16), dpi=100)
gdf.assign(cl=legend.yb).plot(column='cl', categorical=True, k=5, cmap='plasma_r', \
                           linewidth=2, ax=ax, legend=True, legend_kwds={'loc': (0.8,0.75)})
ax.set_axis_off()
texts = ax.get_legend().texts
for txt in texts:
    for k,v in mapping.items():
        if txt.get_text() == str(k):
            txt.set_text(v)
plt.show()